# Testing Algorithm


#### Further References:


In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import sys

import pm4py


sys.path.append("/workspaces/university-petri-nets/")

from backend.compositional_algorithm.combine_nets.combine_nets import MergeNets
from backend.compositional_algorithm.compositional_algorithm import (
    compositional_discovery,
)
from backend.compositional_algorithm.compositional_algorithm import discover
from backend.compositional_algorithm.compositional_algorithm import generate_unique_id
from backend.compositional_algorithm.compositional_algorithm import is_isomorphic
from backend.compositional_algorithm.compositional_algorithm import is_net_valid
from backend.compositional_algorithm.compositional_algorithm import is_refinement
from backend.compositional_algorithm.compositional_algorithm import priority_identifier
from backend.compositional_algorithm.interface_patterns.interface_patterns import (
    INTERFACE_PATTERNS,
)
from backend.compositional_algorithm.transformations.transformations import (
    TRANSFORMATIONS,
)
from backend.compositional_algorithm.transformations.transformations import (
    PlaceTransformation,
)
from backend.compositional_algorithm.transformations.transformations import (
    TransitionTransformation,
)

ImportError: cannot import name 'priority_identifier' from 'backend.compositional_algorithm.compositional_algorithm' (/workspaces/university-petri-nets/backend/compositional_algorithm/compositional_algorithm.py)

#### 0. Algorithm Input


In [ ]:
# define log path
log_path = "/workspaces/university-petri-nets/backend/data_catalog/compositional_process_discovery_experiment_data/IP-1/IP-1_initial_log.xes"

# select algorithm
algorithm = pm4py.discover_petri_net_inductive

# interface pattern
IP1 = INTERFACE_PATTERNS[0]
ip1_net_a1, ip1_initial_marking_a1, ip1_final_marking_a1 = IP1.get_net("A1")
pm4py.view_petri_net(ip1_net_a1, ip1_initial_marking_a1, ip1_final_marking_a1)
ip1_net_a2, ip1_initial_marking_a2, ip1_final_marking_a2 = IP1.get_net("A2")
pm4py.view_petri_net(ip1_net_a2, ip1_initial_marking_a2, ip1_final_marking_a2)
ip1_net = MergeNets.merge_nets(ip1_net_a1, ip1_net_a2)
initial_marking = MergeNets.merge_markings(
    ip1_initial_marking_a1,
    ip1_initial_marking_a2,
)
final_marking = MergeNets.merge_markings(ip1_final_marking_a1, ip1_final_marking_a2)
pm4py.view_petri_net(ip1_net, initial_marking, final_marking)

#### 1. Directly Discover with the given input


In [ ]:
# discover by selected algorithm
net, initial_marking, final_marking = discover(log_path, algorithm)
pm4py.view_petri_net(net, initial_marking, final_marking, format="png")

#### 2. Check Net is valid


In [22]:
# check helper functions works
assert is_net_valid(net, net) is True

#### 3. Check Nets are equal


In [23]:
# check net comparison works
assert is_isomorphic(net, net) is True

#### 4. Check ID Hash


In [24]:
assert generate_unique_id(net) == generate_unique_id(net)

#### 4. Check Priority Identifier


In [ ]:
priority_identifier(net, net)

#### 5. Check is Refinement


##### 5.1 Apply single refinement


In [ ]:
# sequence
transformation_sequence = []

# initial net
copy_refine_net = net.__deepcopy__()

# apply a P1 transformation
places = list(copy_refine_net.places)
P1 = TRANSFORMATIONS[0]
single_refined_net = P1.refine(places[2], copy_refine_net)
transformation_seq_element = (P1, places[2])
transformation_sequence.append(transformation_seq_element)

print(f"refining at place {places[2]}")  # noqa: T201
print(transformation_sequence)  # noqa: T201
pm4py.view_petri_net(single_refined_net)

##### 5.2 Apply multiple refinements


In [ ]:
# Make a deep copy of the original net
multiple_refined_net = ip1_net_a1.__deepcopy__()

# split in place and transition transformations
place_transformations = [
    t for t in TRANSFORMATIONS if isinstance(t(), PlaceTransformation)
]
transition_transformations = [
    t for t in TRANSFORMATIONS if isinstance(t(), TransitionTransformation)
]

# Note: branching logic: we need to apply all possible transformations
# for each place in the current net
for place in multiple_refined_net.places:
    # apply each possible place transformation
    for place_transformation in place_transformations:
        # Note: Deep copy of the current net before applying the transformation -> transformation change places & transitions and sets are immutable.
        net_copy = multiple_refined_net.__deepcopy__()
        transformed_net = place_transformation.refine(place, net_copy)

        # Update multiple_refined_net to the latest transformed net
        multiple_refined_net = transformed_net

# for each transition in the current net
for transition in multiple_refined_net.transitions:
    # apply each possible transition transformation
    for transition_transformation in transition_transformations:
        # Note: ensures that subsequent transformations are applied to a fresh instance of the net.
        net_copy = multiple_refined_net.__deepcopy__()
        transformed_net = transition_transformation.refine(
            transition,
            net_copy,
        )

    # Update multiple_refined_net to the latest transformed net
    multiple_refined_net = transformed_net

pm4py.view_petri_net(multiple_refined_net, initial_marking, final_marking, format="png")

In [ ]:
# check if the net is a refinement
is_ref, path = is_refinement(net, net, TRANSFORMATIONS)
assert is_ref is True

In [ ]:
assert not is_isomorphic(net, single_refined_net)

# check refinement algorithm
is_ref, path = is_refinement(net, single_refined_net, TRANSFORMATIONS)
print("Output:", is_ref, path)  # noqa: T201

In [ ]:
assert not is_isomorphic(net, multiple_refined_net)

# check refinement algorithm
is_ref, path = is_refinement(ip1_net_a1, multiple_refined_net, TRANSFORMATIONS)
print("Output:", is_ref, path)  # noqa: T201

In [ ]:
# Check if refinement is possible
is_ref, path = is_refinement(ip1_net_a1, net, TRANSFORMATIONS)
print(is_ref, path)

#### 4. Check Algorithm


In [ ]:
# discovered net
discovered_net = compositional_discovery(
    input_log_path=log_path,
    algorithm=algorithm,
    interface_pattern=INTERFACE_PATTERNS[0],
    transformations=TRANSFORMATIONS,
)
pm4py.view_petri_net(discovered_net, format="png")

In [ ]:
# compositionally mined process model from paper
pn_coposition_mined, initial_marking, final_marking = pm4py.read_pnml(
    "/workspaces/university-petri-nets/backend/data_catalog/compositional_process_discovery_experiment_data/IP-1/IP-1_composition_mined.pnml",
)
pm4py.view_petri_net(pn_coposition_mined, initial_marking, final_marking, format="png")

#### 5. Conformance Checking


In [ ]:
df_log = pm4py.read_xes(log_path)

In [ ]:
# guarateed by our algorithm
pm4py.check_soundness(net, initial_marking, final_marking)

In [ ]:
# Check conformance of your collaboration process model and the event log with alignment-based fitness
pm4py.fitness_token_based_replay(df_log, net, initial_marking, final_marking)
pm4py.precision_token_based_replay(df_log, net, initial_marking, final_marking)

In [ ]:
# alignment-based precision
pm4py.conformance_diagnostics_alignments(df_log, net, initial_marking, final_marking)

In [ ]:
pm4py.fitness_alignments(df_log, net, initial_marking, final_marking)
pm4py.precision_alignments(df_log, net, initial_marking, final_marking)